## Fetch and preprocess 2D directional lighting images and their corresponding height data to create dataset

In [ ]:
%load_ext autoreload
%autoreload 2

# External
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import pickle as pkl
import os
import numpy as np
from PIL import Image, ImageOps
import csv
from tqdm import tqdm

### Loop through and fetch, preprocess, and save sample's data

In [ ]:
for i in tqdm(range(len(samples))):
    sample = samples[i]

    # <-------------- Fetch -------------->
    # redacted code to fetch a sample's directional lighting images

    # <-------------- Download -------------->
    # ensure downloads exist
    if height_data is None:
        failed_samples += [(sample, side, "download failed")]
        break
    if heightmap_mask is None:
        failed_samples += [(sample, side, "download failed")]
        break
    if north_image is None:
        failed_samples += [(sample, side, "download failed")]
        break
    if east_image is None:
        failed_samples += [(sample, side, "download failed")]
    if south_image is None:
        failed_samples += [(sample, side, "download failed")]
    if west_image is None:
        failed_samples += [(sample, side, "download failed")]

    # <-------------- Match Input Directional Lighting Image Sizes to North Image's Size -------------->
    # resize all directional images to north image's shape
    north_size = list(reversed(north_image.shape[:2]))
    east_image = np.array(ImageOps.fit(Image.fromarray(east_image), north_size, Image.ANTIALIAS))
    south_image = np.array(ImageOps.fit(Image.fromarray(south_image), north_size, Image.ANTIALIAS))
    west_image = np.array(ImageOps.fit(Image.fromarray(west_image), north_size, Image.ANTIALIAS))

    # <-------------- Use North Image's Mask to Keep ROI -------------->
    # obtain North image's mask
    north_mask = get_mask(north_image)  # use U-Net model for mask finding

    # rescale to 0 to 1
    north_mask = (north_mask / 255).astype('uint8')

    # only keep film (discard tab)
    north_image = north_image * north_mask
    east_image = east_image * north_mask
    south_image = south_image * north_mask
    west_image = west_image * north_mask

    # <-------------- Crop All Images to their Minimum Bounding Rectangle -------------->
    # crop directional lighting images to their minimum bounding rectangle
    x, y, w, h = cv2.boundingRect(north_mask)
    north_image = north_image[y:y+h, x:x+w]
    east_image = east_image[y:y+h, x:x+w]
    south_image = south_image[y:y+h, x:x+w]
    west_image = west_image[y:y+h, x:x+w]

    # crop height data to its minimum bounding rectangle
    x, y, w, h = cv2.boundingRect(heightmap_mask)
    height_data = height_data[y:y+h, x:x+w]

    # <-------------- Match Directional Lighting Image Sizes to Height Data's Size -------------->
    # use cropped height image as target dimensions
    crop_height_image_size = tuple(reversed(height_data.shape))

    # resize directional images to target cropped height image dimensions
    north_image = np.array(ImageOps.fit(Image.fromarray(north_image), crop_height_image_size, Image.ANTIALIAS))
    east_image = np.array(ImageOps.fit(Image.fromarray(east_image), crop_height_image_size, Image.ANTIALIAS))
    south_image = np.array(ImageOps.fit(Image.fromarray(south_image), crop_height_image_size, Image.ANTIALIAS))
    west_image = np.array(ImageOps.fit(Image.fromarray(west_image), crop_height_image_size, Image.ANTIALIAS))

    # <-------------- Subtract 2nd Order Fit From Height Data -------------->
    curved_image = height_util.best_fit_polynomial(height_data, order=2)
    fit_height_data = height_data - curved_image

    # <-------------- Constrain All Images to be (1723, 1402, n) -------------->
    final_size = (1723, 1402)

    # resize height data to final target dimensions
    height_data = np.array(ImageOps.fit(Image.fromarray(height_data), final_size, Image.ANTIALIAS))
    fit_height_data = np.array(ImageOps.fit(Image.fromarray(fit_height_data), final_size, Image.ANTIALIAS))

    # resize directional images to final target dimensions
    north_image = np.array(ImageOps.fit(Image.fromarray(north_image), final_size, Image.ANTIALIAS))
    east_image = np.array(ImageOps.fit(Image.fromarray(east_image), final_size, Image.ANTIALIAS))
    south_image = np.array(ImageOps.fit(Image.fromarray(south_image), final_size, Image.ANTIALIAS))
    west_image = np.array(ImageOps.fit(Image.fromarray(west_image), final_size, Image.ANTIALIAS))

    if height_data.shape != (1402, 1723) \
    or north_image.shape != (1402, 1723) \
    or east_image.shape != (1402, 1723) \
    or south_image.shape != (1402, 1723) \
    or west_image.shape != (1402, 1723):
        print((sample, side, "dim not 1402 x 1723"))
        failed_samples += [(sample, side, "dim not 1402 x 1723")]
        continue

    # <-------------- Create an Input Tensor -------------->
    # create tensor of 4, grayscale, directional VCB images
    input_tensor = np.zeros((1402, 1723, 4))
    input_tensor[:, :, 0] = north_image
    input_tensor[:, :, 1] = east_image
    input_tensor[:, :, 2] = south_image
    input_tensor[:, :, 3] = west_image

    # <-------------- Save Input Tensor and Height Data -------------->
    np.save(f"./grayscale_input/{sample}_{side}_input_tensor.npy", input_tensor.astype('uint8'))
    np.save(f"./height_data/{sample}_{side}_height_data.npy", height_data)
    np.save(f"./quadratic_fit_height_data/{sample}_{side}_height_data.npy", fit_height_data)

    # <-------------- Update dataset.csv With New Pair -------------->
    with open('dataset.csv', 'a') as data_file:    
        write = csv.writer(data_file)
        write.writerow([f"{sample}_input_tensor.npy", f"{sample}_height_data.npy"])
        data_file.close()